## 9. High Accuracy CNN for MNIST
_Exercise: Build your own CNN from scratch and try to achieve the highest possible accuracy on MNIST._

The following model uses 

- 2 convolutional layers, 

- followed by 1 pooling layer, 

- then dropout 25%, 

- then a dense layer, 

- another dropout layer but with 50% dropout, 

- and finally the output layer. 

It reaches about 99.2% accuracy on the test set. This places this model roughly in the top 20% in the [MNIST Kaggle competition](https://www.kaggle.com/c/digit-recognizer/) (if we ignore the models with an accuracy greater than 99.79% which were most likely trained on the test set, as explained by Chris Deotte in [this post](https://www.kaggle.com/c/digit-recognizer/discussion/61480)). 

Can you do better? 

To reach 99.5 to 99.7% accuracy on the test set, you need to add 

- image augmentation, 

- batch norm, use a learning schedule such as 1-cycle, 

- and possibly create an ensemble.

In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

In [10]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()

X_train_full = X_train_full / 255
X_test = X_test / 255

X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [11]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [12]:
# Model building
model = keras.models.Sequential([
    keras.layers.Conv2D(32, kernel_size=3, padding="same", activation="relu"),
    keras.layers.Conv2D(64, kernel_size=3, padding="same", activation="relu"),
    keras.layers.MaxPool2D(),
    keras.layers.Flatten(),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])

In [13]:
# Model training and testing
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
model.evaluate(X_test, y_test)

Epoch 1/10
1719/1719 [==============================] - 140s 80ms/step - loss: 0.1922 - accuracy: 0.9428 - val_loss: 0.0500 - val_accuracy: 0.9860
Epoch 2/10
1719/1719 [==============================] - 133s 77ms/step - loss: 0.0812 - accuracy: 0.9751 - val_loss: 0.0444 - val_accuracy: 0.9884
Epoch 3/10
1719/1719 [==============================] - 127s 74ms/step - loss: 0.0637 - accuracy: 0.9806 - val_loss: 0.0356 - val_accuracy: 0.9914
Epoch 4/10
1719/1719 [==============================] - 129s 75ms/step - loss: 0.0474 - accuracy: 0.9853 - val_loss: 0.0342 - val_accuracy: 0.9912
Epoch 5/10
1719/1719 [==============================] - 124s 72ms/step - loss: 0.0429 - accuracy: 0.9867 - val_loss: 0.0328 - val_accuracy: 0.9908
Epoch 6/10
1719/1719 [==============================] - 125s 73ms/step - loss: 0.0382 - accuracy: 0.9877 - val_loss: 0.0333 - val_accuracy: 0.9924
Epoch 7/10
1719/1719 [==============================] - 126s 73ms/step - loss: 0.0319 - accuracy: 0.9900 - val_loss: 0

[0.031391628086566925, 0.9911999702453613]

## 10.  Use transfer learning for large image classification

_Exercise: Use transfer learning for large image classification, going through these steps:_

* _Create a training set containing at least 100 images per class. For example, you could classify your own pictures based on the location (beach, mountain, city, etc.), or alternatively you can use an existing dataset (e.g., from TensorFlow Datasets)._
* _Split it into a training set, a validation set, and a test set._
* _Build the input pipeline, including the appropriate preprocessing operations, and optionally add data augmentation._
* _Fine-tune a pretrained model on this dataset._

In [19]:
import tensorflow_datasets as tfds

ModuleNotFoundError: No module named 'tensorflow_datasets'

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load the dataset
dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)

# Define the classes
num_classes = info.features['label'].num_classes

# Define the training, validation, and test split sizes
train_split = 0.7
val_split = 0.15
test_split = 0.15

# Calculate the number of examples for each split
num_examples = info.splits['train'].num_examples
num_train_examples = int(train_split * num_examples)
num_val_examples = int(val_split * num_examples)
num_test_examples = num_examples - num_train_examples - num_val_examples

In [ ]:
# Split the dataset
train_dataset = dataset['train'].take(num_train_examples)
val_dataset = dataset['train'].skip(num_train_examples).take(num_val_examples)
test_dataset = dataset['train'].skip(num_train_examples + num_val_examples).take(num_test_examples)

In [ ]:
# Define preprocessing function
def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224))  # Resize image to expected input size for MobileNetV2
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)  # Preprocess the input image
    return image, label

# Apply preprocessing to datasets
train_dataset = train_dataset.map(preprocess_image)
val_dataset = val_dataset.map(preprocess_image)
test_dataset = test_dataset.map(preprocess_image)

# Define batch size
batch_size = 32

# Shuffle and batch the datasets
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(batch_size)
val_dataset = val_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

In [ ]:
# Load pretrained MobileNetV2 model
pretrained_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pretrained layers
pretrained_model.trainable = False

In [ ]:
# Define the new classification head
model = tf.keras.Sequential([
    pretrained_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_dataset)
print("Test Accuracy:", test_acc)